![Image Alt Text](https://superdesk-pro-c.s3.amazonaws.com/sd-nepalitimes/20221202151240/638a0e819c7e80680e0cd473jpeg.jpg)


In [ ]:
import cv2
import os
import shutil
import random

import pandas as pd

import matplotlib.pyplot as plt
import numpy as np

import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.applications import Xception
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam

from keras.layers import Dense,Dropout,Flatten,BatchNormalization,MaxPooling2D,Conv2D,Input, GlobalAveragePooling2D
from keras.callbacks import EarlyStopping,ModelCheckpoint

import keras_tuner as kt

from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, classification_report

# Part 1: Converting videos to images

In [ ]:
data_dir = './data'
os.makedirs(data_dir, exist_ok=True)

In [ ]:
traffic_unrelated = f'{data_dir}/traffic_unrelated'
light_traffic = f'{data_dir}/light_traffic'
moderate_traffic = f'{data_dir}/moderate_traffic'
heavy_traffic = f'{data_dir}/heavy_traffic'
congested_traffic = f'{data_dir}/congested_traffic'

# Creating directories if they don't exist
os.makedirs(traffic_unrelated, exist_ok=True)
os.makedirs(light_traffic, exist_ok=True)
os.makedirs(moderate_traffic, exist_ok=True)
os.makedirs(heavy_traffic, exist_ok=True)
os.makedirs(congested_traffic, exist_ok=True)

In [ ]:
def convert_video_to_frames(video_path, output_folder):

    # Open the video file
    cap = cv2.VideoCapture(video_path)

    frame_count = 0

    # Loop through the video frames
    while True:
        ret, frame = cap.read()

        # If no more frames are available, break the loop
        if not ret:
            break

        # Construct the output file path for the frame
        frame_filename = os.path.join(output_folder, f"{output_folder[0]}_frame_{frame_count:04d}.jpg")

        # Save the frame as a JPEG image
        cv2.imwrite(frame_filename, frame)

        frame_count += 1

    # Release the video file and close the output folder
    cap.release()

    print(f"Converted {frame_count} frames to JPG images.")

In [ ]:
convert_video_to_frames(
    video_path = "/kaggle/input/traffic-congestion-videos/Non-traffic related.mp4", 
    output_folder = traffic_unrelated
)

In [ ]:
convert_video_to_frames(
    video_path = "/kaggle/input/traffic-congestion-videos/Light Traffic.mp4", 
    output_folder = light_traffic
)

In [ ]:
convert_video_to_frames(
    video_path = "/kaggle/input/traffic-congestion-videos/Moderate Traffic.mp4", 
    output_folder = moderate_traffic
)

In [ ]:
convert_video_to_frames(
    video_path = "/kaggle/input/traffic-congestion-videos/Heavy Traffic.mp4", 
    output_folder = heavy_traffic
)

In [ ]:
convert_video_to_frames(
    video_path = "/kaggle/input/traffic-congestion-videos/Congested Traffic.mp4", 
    output_folder = congested_traffic
)

# Part 2: Splitting Images

In [ ]:
train_dir = 'train'
valid_dir = 'valid'

# Creating directories if they don't exist
os.makedirs(train_dir, exist_ok=True)
os.makedirs(valid_dir, exist_ok=True)

In [ ]:
class1 = 'traffic_unrelated'
class2 = 'light_traffic'
class3 = 'moderate_traffic'
class4 = 'heavy_traffic'
class5 = 'congested_traffic'

classes = [class1, class2, class3, class4, class5]

for each in classes:
    os.makedirs(os.path.join(train_dir, each), exist_ok=True)
    os.makedirs(os.path.join(valid_dir, each), exist_ok=True)

In [ ]:
def split_data(class_dir, train_ratio, valid_ratio):
    files = os.listdir(class_dir)
    random.shuffle(files)

    num_files = len(files)
    train_split = int(train_ratio * num_files)

    train_files = files[:train_split]
    valid_files = files[train_split:]
    
    random.shuffle(train_files)
    random.shuffle(valid_files)
    
    return train_files, valid_files

# Specifying the split ratios
train_ratio = 0.85
valid_ratio = 0.15

split_info = {}

for each in classes:
    
    train, validation = split_data(os.path.join(data_dir, each), train_ratio, valid_ratio)
    
    train_size, validation_size = len(train), len(validation)
    split_info[each] = [train_size, validation_size]
    
    # Copying files to their respective directories
    for file in train:
        shutil.copy(os.path.join(data_dir, each, file), os.path.join(train_dir, each, file))

    for file in validation:
        shutil.copy(os.path.join(data_dir, each, file), os.path.join(valid_dir, each, file))

In [ ]:
split_info

In [ ]:
categories = split_info.keys()
train_counts = [item[0] for item in split_info.values()]
validation_counts = [item[1] for item in split_info.values()]

# Bar width
bar_width = 0.35

# Bar positions
train_positions = range(len(categories))
validation_positions = [pos + bar_width for pos in train_positions]

fig, ax = plt.subplots(figsize=(10, 5))

# Create the bar plot
plt.bar(train_positions, train_counts, bar_width, label='Train')
plt.bar(validation_positions, validation_counts, bar_width, label='Validation')

# Set the x-axis labels and rotate them
plt.xticks([pos + bar_width / 2 for pos in train_positions], categories, rotation=45)

# Set labels and title
plt.xlabel('Traffic Categories')
plt.ylabel('Count of Images')
plt.title('Dataset Training and Validation Image Counts by Traffic Category')

# Display a legend
plt.legend()

# Annotate the bars with their values
for i, v in enumerate(categories):
    plt.text(train_positions[i] + bar_width / 2, train_counts[i] + 5, str(train_counts[i])
             , ha='center', va='bottom')
    plt.text(validation_positions[i] + bar_width / 2, validation_counts[i] + 5, str(validation_counts[i]
             ), ha='center', va='bottom')

plt.grid(axis='y')
# Show the plot
plt.show()

# Part 3: Deleteing Previous Images

In [ ]:
def delete_non_empty_directory(directory_path):
    try:
        # Use shutil.rmtree to delete the directory and its contents
        shutil.rmtree(directory_path)
        print(f"Successfully deleted the directory: {directory_path}")
    except Exception as e:
        print(f"Error: {e}")

In [ ]:
delete_non_empty_directory(data_dir)

# Part 4: Image Batches Generators + Real-Time Data Augmentation/

In [ ]:
img_width, img_height = 300, 300
batch_size = 256

In [ ]:
train_datagen = ImageDataGenerator( 
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.1,
    brightness_range=[0.5, 1.25],
    horizontal_flip=True,
    vertical_flip=True
)
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    seed=55
)

In [ ]:
validation_datagen = ImageDataGenerator()
validation_generator = validation_datagen.flow_from_directory(
    valid_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    seed=44
)

In [ ]:
print(train_generator.class_indices)
print(validation_generator.class_indices)

# Part 5: Hyperparameter Tuning

In [ ]:

def model_builder(hp):
    
    
    base_model = Xception(
        input_shape=(img_width, img_height, 3),
        weights='imagenet', 
        include_top=False, 
        pooling=hp.Choice('base_model_pooling', values=[ 'avg', 'max' ]) 
    )  
    base_model.trainable = False
    
    
    i = Input([img_width, img_height, 3], dtype = tf.uint8)
    x = tf.cast(i, tf.float32)
    x = tf.keras.applications.xception.preprocess_input(x)
    x = base_model(x, training=False)  
    x = BatchNormalization()(x)
    x = Dense( hp.Int('units_1', min_value=16, max_value=512, step=16), activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dense( hp.Int('units_2', min_value=16, max_value=512, step=16), activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout( hp.Float('dropout_rate_1', min_value=0.1, max_value=0.6, step=0.1))(x)
    x = Dense(5, activation='softmax')(x) 
    model = tf.keras.Model(inputs=[i], outputs=[x])

    
    model.compile(
        loss = 'categorical_crossentropy', 
        metrics = ['accuracy'],
        optimizer = Adam(
            lr = hp.Choice('learning_rate', values=[0.003, 0.001, 0.0007, 0.0003, 0.0001])
        )
    )
    
    
    return model



tuner = kt.Hyperband(model_builder,
                     objective='val_loss',
                     max_epochs=10,
                     factor=3,
                     directory='/kaggle/working/',
                     project_name='hyperparameter_tuning_traffic_classifier_model_0')
   
    
    
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)



tuner.search(
    train_generator,
    steps_per_epoch=(train_generator.samples // batch_size) // 3, #  using only the first half of the train dataset
    validation_data=validation_generator,
    validation_steps=(validation_generator.samples // batch_size)  // 3, #  using only the first half of the validation dataset
    batch_size=batch_size,
    callbacks=[stop_early]
)

In [ ]:
print(tuner.get_best_hyperparameters(num_trials=1)[0].get('base_model_pooling'))
print(tuner.get_best_hyperparameters(num_trials=1)[0].get('units_1'))
print(tuner.get_best_hyperparameters(num_trials=1)[0].get('units_2'))
print(tuner.get_best_hyperparameters(num_trials=1)[0].get('dropout_rate_1'))
print(tuner.get_best_hyperparameters(num_trials=1)[0].get('learning_rate')) 

# Part 6: Model with optimum hyperparameter

In [ ]:
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]
model = tuner.hypermodel.build(best_hps)
model.summary()

In [ ]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size, #  using the full train dataset
    epochs=51, 
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size, #  using the full validation dataset
    batch_size=batch_size
)

# Part 7: Validating Models

In [ ]:
def print_evaluation_metrics(model, generator, batch_size):
    X, y = [], []

    # Reset the generator
    generator.reset()

    # Loop through the generator to retrieve images and labels
    for i in range((generator.samples // batch_size)+1):
        batch_x, batch_y = generator.next()
        X.extend(batch_x)
        y.extend(batch_y)

    # Convert X and y to numpy arrays
    X = np.array(X)
    y = np.array(y)
    
    predictions = model.predict(X, verbose=1)
    y_pred = np.argmax(predictions, axis=1)
    
    y_true = np.argmax(y, axis=1)
    
    print()

    conf_matrix = confusion_matrix(y_true, y_pred)
    print("Confusion Matrix:")
    print(conf_matrix)
    print()

    class_report = classification_report(y_true, y_pred)
    print("Classification Report:")
    print(class_report)

    confusion_matrix(y_true, y_pred)

In [ ]:
score, acc = model.evaluate(validation_generator)
print('Test Loss =', score)
print('Test Accuracy =', acc)

In [ ]:
print_evaluation_metrics(model, validation_generator, batch_size)

In [ ]:
score, acc = model.evaluate(train_generator)
print('Test Loss =', score)
print('Test Accuracy =', acc)

In [ ]:
print_evaluation_metrics(model, train_generator, batch_size)

In [ ]:
hist_=pd.DataFrame(history.history)

plt.figure(figsize=(15,5))
plt.subplot(1,2,1)
plt.plot(hist_['loss'],label='Train_Loss')
plt.plot(hist_['val_loss'],label='Validation_Loss')
plt.title('Train_Loss & Validation_Loss',fontsize=20)
plt.legend()
plt.subplot(1,2,2)
plt.plot(hist_['accuracy'],label='Train_Accuracy')
plt.plot(hist_['val_accuracy'],label='Validation_Accuracy')
plt.title('Train_Accuracy & Validation_Accuracy',fontsize=20)
plt.legend()
plt.show()

# Part 8: Saving best model

In [ ]:
model.save('traffic_classifier.h5')

# Download Model

In [ ]:
from IPython.display import FileLink

print('Click below to download model:')
# Generate a download link
FileLink('traffic_classifier.h5')